In [2]:
import matplotlib.pyplot as plt
import numpy as np
import nltk
import sklearn
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.datasets import load_files
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten

# fix random seed for reproducibility
np.random.seed(7)

In [3]:
#load training files
path_train = '//bbc/train'
reviewsTrain = load_files(path_train)
X_train, y_train = reviewsTrain.data, reviewsTrain.target

#load test files
path_test = '//bbc/test'
reviewsTest = load_files(path_test)
X_test, y_test = reviewsTest.data, reviewsTest.target

FileNotFoundError: [WinError 53] The network path was not found: '//bbc/train'

In [ ]:
# encoding traing data into UTF-8 
textTrain = []
for each in X_train:
    x = each.decode("UTF-8", errors = 'ignore')
    textTrain.append(x)

# converting the training labels into numbers
labelsTrain = np.array(y_train)
labelsTrain_one_hot = pd.get_dummies(labelsTrain)

# encoding test data into UTF-8
textTest = []
for each in X_test:
    y = each.decode("UTF-8", errors = 'ignore')
    textTest.append(y)
    
# converting the training labels into numbers   
labelsTest = np.array(y_test)
labelsTest_one_hot = pd.get_dummies(labelsTest)

In [ ]:
# tokenizing the training and test data and remove stop words
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

for doc in range(len(textTrain)):
    clean_text_train = []
    text = (textTrain[doc])
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    for token in tokens:
        if not token in stop_words:
            clean_text_train.append((token))
    news = " ".join(clean_text_train)
    textTrain[doc] = news
    
for doc in range(len(textTest)):
    clean_text_test = []
    text = (textTest[doc])
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    for token in tokens:
        if not token in stop_words:
            clean_text_test.append((token))
    news = " ".join(clean_text_test)
    textTest[doc] = news
    
print("documents in training dataset:", len(textTrain))
print("documents ib training dataset:",len(textTest))

In [ ]:
# use only top 10000 words for embedding layer
top_words = 10000
t = Tokenizer(num_words = top_words)
t.fit_on_texts(textTrain)
vocab_size = len(t.word_index) + 1
print(vocab_size)
encoded_docs = t.texts_to_sequences(textTrain)
#print(encoded_docs)
# use only 3000 words from each article and if the article is shorter than 3000 words, do padding (post)
max_length = 3000
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)

In [8]:
# embedding vector length is 300
embedding_vecor_length = 300
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_length))
model.add(Dropout(0.1))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3000, 300)         3000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 3000, 300)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 505       
Total params: 3,160,905
Trainable params: 3,160,905
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# measure the time, loss and accuracy at each epoch
import datetime
import time
start_time = datetime.datetime.now().time().strftime('%H:%M:%S')
model.fit(padded_docs, labelsTrain_one_hot, epochs=3)
end_time = datetime.datetime.now().time().strftime('%H:%M:%S')
total_time=(datetime.datetime.strptime(end_time,'%H:%M:%S') - datetime.datetime.strptime(start_time,'%H:%M:%S'))
print(total_time)

Epoch 1/3
1584/1584 [==============================] - 446s 282ms/step - loss: 0.5262 - acc: 0.7895
Epoch 2/3
1584/1584 [==============================] - 426s 269ms/step - loss: 0.5035 - acc: 0.8000
Epoch 3/3
1584/1584 [==============================] - 441s 278ms/step - loss: 0.5022 - acc: 0.8000
0:21:56


In [10]:
# time to test our model
# converting test dataset into integers and then padding
top_words = 10000
t = Tokenizer(num_words = top_words, lower = True)
t.fit_on_texts(textTest)
vocab_size = len(t.word_index) + 1
#print(vocab_size)
encoded_docs = t.texts_to_sequences(textTest)

max_length = 3000
padded_test = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)
# accuracy score of the model is 80%
scores = model.evaluate(padded_test, labelsTest_one_hot, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))